<center>Progetto realizzato da Elena Curti (matr. 185431)

# Recipes
</center>

## Requisiti
Questo progetto è stato realizzato con Python 3.11 e Neo4j. E' necessaria l'installazione dei seguenti pacchetti:

In [1]:
# %pip install py2neo neo4j-driver

## Operazioni iniziali
Per il corretto funzionamento del progetto proposto, è necessario creare il DBMS e importarci i dati, seguendo i seguenti passi. <br>
Da Neo4j creare un nuovo progetto. Scegliere una tra le seguenti alternative:
- Dal progetto appena creato, aggiungere al progetto il file [neo4j.dump](data/neo4j.dump). Creare poi un nuovo DBMS dal file dump (scegliendo come nome "neo4j", come password "pass" e come versione 5.2.0). Aggiungere il plugin APOC. Premere su Start.
- Dal progetto appena creato, aggiungere un DBMS locale (scegliendo come nome, password e versione i valori indicati al punto sopra). Aggiungere il plugin APOC. Premere su Start e poi su Open. Eseguire tutti i comandi riportati nel file [import_commands.txt](data/import_commands.txt).
<br>
Eseguire poi le celle di codice, nell'ordine proposto

## Connessione e autenticazione

In [2]:
from py2neo import Graph
graph = Graph("bolt://localhost:7687",  auth=("neo4j", "pass"))

print("Connessione al database riuscita! \nEcco 5 ricette d'esempio:")
cq = "MATCH (p:Recipe) RETURN p.name LIMIT 5"
for rec in graph.run(cq):
    print("\t",str(rec)[1:-1])

Connessione al database riuscita! 
Ecco 5 ricette d'esempio:
	 Tomato & mozzarella couscous salad
	 Baked nectarines & raspberries with almonds & honey
	 Chocolate & caramel ombre cake
	 Chicken, goat's cheese & cherry tomato bake
	 Chipotle chicken tacos with pineapple salsa


# Schema iniziale
Lo use case originale è stato trovato sul sito [Recipes Neo4j](https://neo4j.com/graphgists/dd3dedcf-c377-4575-84f4-4d0d30b2a4c5/). Ha la seguente struttura iniziale:
```
    call db.schema.visualization()
```

![schema](img/db_schema2.png)

I nodi Ingredient, Author, Collection, DietType, Keyword ha un solo campo "name" che rappresenta il nome dell'ingrediente. <br>
Il nodo Recipe ha le seguenti proprietà:
<ul>
    <li>id: campo che identifica univocamente la ricetta </li>
    <li>cookingTime: tempo di cottura </li>
    <li>description: descrizione della ricetta </li>
    <li>name: nome della ricetta </li>
    <li>preparationTime: tempo di preparazione </li>
    <li>skillLevel: difficoltà </li>
</ul>
Gli archi non hanno proprietà.

## Aggiunta di dati e modifiche allo schema originale
Per arricchire il progetto ho deciso di aggiungere dei dati presi da Kaggle e di eseguire le seguenti modifiche allo schema originale. 
<br>
[OK] Aggiunto il campo is_allergene negli ingredienti. (TODO andrà messo dopo)<br>
[ok] Rimosse le keyword<br>
Aggiunta la proprieta persone_pezzi, link, bbc_or_gf nella ricetta<br>
aggiunto campo quantita in contains_ingredients<br>
aggiunto campo  <br>

<MARK> TODO ARRIVATA QUI  -> Aggiungere https://www.kaggle.com/datasets/edoardoscarpaci/italian-food-recipes

### 0. Copiamo i files necessari 
Prima di effettuare modifiche, copio i file che verranno usati nella cartella di Neo4j con gli import

In [3]:
cq ="""CALL dbms.listConfig() YIELD name, value
WHERE name="server.directories.import"
RETURN value"""
import os 
DATA_FOLDER ="data"+os.path.sep
dir_import = graph.run(cq).evaluate()
# print(type(dir_import))
print(dir_import)

import shutil
for file_name in os.listdir(DATA_FOLDER):
    # construct full file path
    source = DATA_FOLDER + file_name
    destination = dir_import + os.path.sep + file_name
    if os.path.isfile(source):
        shutil.copy(source, destination)
        #print('copied', file_name)



D:\Elena\_Elena\Shared\Universita\Magistrale\Big_data\Anno_22_23\TO_DEL_QUANDO_ESAME_DATO\Neo4J\relate-data\dbmss\dbms-bcc1bfb5-d8f3-420f-9a93-21eedafd8b70\import


### 1. Ingredienti
Ho aggiunto un campo "is_allergene" al nodo Ingredient. Ho inizialmente scaricato da https://www.kaggle.com/datasets/boltcutters/food-allergens-and-allergies un file ([FoodData.csv](data/FoodData.csv)) con l'elenco degli allergeni. Ho poi impostato gli ingredienti contenuti nel file con is_allergene=True. Ho poi settato is_allergene=False per i restanti ingredienti. 

In [4]:
IMPORT_FOLDER=r"D:\Elena\_Elena\Shared\Universita\Magistrale\Big_data\Anno_22_23\TO_DEL_QUANDO_ESAME_DATO\Neo4J\relate-data\dbmss\dbms-4f6affc1-7465-4512-8d5d-bf6bd4552aa3\import"

In [5]:
# Setto gli allergeni
cq="""LOAD CSV FROM 'file:///FoodData.csv' AS value
MATCH (i:Ingredient)
WHERE toLower(i.name) CONTAINS toLower(value[3]) AND value[3]<>"Food"
SET i.is_allergene=True;"""
graph.run(cq)

# Setto i restanti ingredienti come non allergeni
cq="""
MATCH (i:Ingredient)
WHERE i.is_allergene IS NULL
SET i.is_allergene=False; """
graph.run(cq)


(No data)

### 2. Rimuovo il nodo Keyword


In [6]:
cq="""
MATCH (n:Keyword) DETACH DELETE n
"""
graph.run(cq)

(No data)

Copio il file appena creato nella cartella import 

## Aggiunta degli indici
Per migliorare la performance delle cypher query sono stati aggiunti i seguenti indici: 
    <ul>
        <li><i>id</i> su Recipe</li>
        <li><i>name</i> su Ingredient</li>
        <li><i>name</i> su Keyword</li>
        <li><i>name</i> su DietType</li>
        <li><i>name</i> su Author</li>
        <li><i>name</i> su Collection</li>
    </ul>
Per creare gli indici è stato usato il seguente script. Il comando ```CREATE INDEX IF NOT EXISTS FOR (n:Recipe) ON (n.id)``` è equivalente a ```CREATE INDEX IF NOT EXISTS ON :Recipe(name)```. Il primo, però, è compatibile con la versione 5.2.0 di Neo4j.

In [7]:
def run_multiple_query(multiple_query):
    for query in multiple_query.splitlines():
        if query == "": continue
        graph.run(query)

cq_versione5= """
CREATE INDEX IF NOT EXISTS FOR (n:Recipe) ON (n.id);        // CREATE INDEX IF NOT EXISTS ON :Recipe(name) 
CREATE INDEX IF NOT EXISTS FOR (n:Ingredient) ON (n.name);
CREATE INDEX IF NOT EXISTS FOR (n:Keyword) ON (n.name);
CREATE INDEX IF NOT EXISTS FOR (n:DietType) ON (n.name);
CREATE INDEX IF NOT EXISTS FOR (n:Author) ON (n.name);
CREATE INDEX IF NOT EXISTS FOR (n:Collection) ON (n.name);
""" 
run_multiple_query(cq_versione5)

In [8]:
cq = "MATCH (d:Recipe) RETURN d"
# print(cq)
# for i in graph.run(cq):
#     print(i)